In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Чтение датасета
df = pd.read_csv('../../data/_data.csv')

In [20]:
# удаляем столбец `Unnamed: 0`
df = df.iloc[:, 1:]

# переименовываем столбцы
NEW_COLUMNS = {
    'ID  объявления':           'id',
    'Количество комнат':        'rooms',
    'Тип':                      'type',
    'Метро':                    'metro_station',
    'Адрес':                    'address',
    'Площадь, м2':              'area',
    'Дом':                      'building_type',
    'Парковка':                 'parking',
    'Цена':                     'price',
    'Телефоны':                 'phones',
    'Описание':                 'description',
    'Ремонт':                   'renovation',
    'Площадь комнат, м2':       'rooms_area',
    'Балкон':                   'balcony',
    'Окна':                     'windows',
    'Санузел':                  'bathroom',
    'Можно с детьми/животными': 'kids_pets',
    'Дополнительно':            'extra',
    'Название ЖК':              'complex_title',
    'Серия дома':               'building_series',
    'Высота потолков, м':       'ceiling_height',
    'Лифт':                     'elevator',
    'Мусоропровод':             'trash_chute',
    'Ссылка на объявление':     'listing_url',
}
df = df.rename(columns=NEW_COLUMNS)

In [ ]:
# 1. Создаем столбец с главной ценой
# r'([\d\.]+)' Это регулярное выражение, которое используется для поиска чисел с точкой. 
# \d — означает любую цифру от 0 до 9. 
# [\d\.]+ — это: «одна или больше цифр и/или точек подряд». 
df['main_price'] = df['Цена'].str.extract(r'([\d\.]+)').astype(float)

# 2. Поработаем с залогом
# Ищем слово Залог, дефис, и число после него
# *-\s* это мы так страхуемся, чтобы захватились пробелы до и после дефиса"""
df['deposit'] = df['Цена'].str.extract(r'Залог\s*-\s*([\d\s]+)')[0]
df['deposit'] = df['deposit'].str.replace(' ', '').astype(float)

# 3. Коммунальные услуги - выводим пока что только булевы значения True/False
df['utilities_included'] = df['Цена'].str.contains('Коммунальные услуги включены', case=False, na=False)

# 4. Срок аренды
# Ищем слово Срок аренды, дефис, и слова после него
df['rental_term'] = df['Цена'].str.extract(r'Срок аренды\s*-\s*([а-яА-Яa-zA-Z ]+)')

# 5. Предоплата
# Ищем слово Предоплата, дефис, и слова после него
df['prepayment_months'] = df['Цена'].str.extract(r'Предоплата\s*(\d+)')
df['prepayment_months'] = df['prepayment_months'].astype(float)

df.head(5)

In [ ]:
df['num_rooms'] = df['rooms'].str.extract(r'(\d+)')
df['num_rooms'] = df['num_rooms'].fillna(df['num_rooms'].dropna().astype('int64').median()).astype('int64')
df['num_rooms'].unique()

In [ ]:
df['type_rooms'] = df['rooms'].str.extract(r'(\s\D+)')
df['type_rooms'].unique()
